# Digital phantom

In [83]:
import pydicom
import numpy as np
import csv
import pandas as pd

The dataset Digital phantom has the following caracteristics :
- The phantom consists of 4×4x5 (x,y,z) voxels.
- The voxels in the ROI (Region Of Interest) are defined in the segmentation mask (their values equals 1).
- The intensities of each voxel of the ROI is defined in the image.  

First, I have to read the dicom files of the segmentation mask and the image, then I'll get the set of intensities of voxels included in the ROI. Once this set ready, I can compute the intensity-based statistical features

In [90]:
#Read the dicom file of the segmentation mask
mask = pydicom.read_file("ibsi_1_digital_phantom/dicom/mask/mask.dcm")
#Get the voxels' matrix defined by the segmentation mask
maskvox = np.zeros((4,4,5))
maskvox = mask.pixel_array

#Read the dicom file of the image
image = pydicom.read_file("ibsi_1_digital_phantom/dicom/image/phantom.dcm")
#Get the set of intensities of the voxels
imageintes = np.zeros((4,4,5))
imageintens = image.pixel_array


#Build the matrix of intensities considering only the voxels defined by the segmentation mask
intensities = np.zeros((4,4,5))
intensities = np.where(maskvox==1,imageintens,maskvox)

#Get the number of voxels with intensity 1 in the mask segmentation
count = (maskvox == 1).sum()


Here, I will define the functions which compute the eighteen "intensity-based statistical features". Each function compute a single feature.

In [91]:
def mean_intensity(intensities, count):
    return (1/count)*np.sum(intensities)

def intesity_variance(intensities,count):
    #Build the centered data matrix of intensities
    intensitiescent = np.where(intensities==0,intensities,intensities-mean_intensity(intensities, count))
    return ((np.sum((intensitiescent)**2))*(1/count))

def intensity_skewness(intensities,count):
    #Build the centered data matrix of intensities
    intensitiescent = np.where(intensities==0,intensities,intensities-mean_intensity(intensities, count))
    return ((np.sum((intensitiescent)**3))*(1/count))/(intesity_variance(intensitiescent,count)**(3/2))

def intensity_kurtosis(intensities,count):
    #Build the centered data matrix of intensities
    intensitiescent = np.where(intensities==0,intensities,intensities-mean_intensity(intensities, count))
    return (((np.sum((intensitiescent)**4))*(1/count))/(intesity_variance(intensitiescent,count)**2))-3

def median_intensity(intensities):
    return np.median(intensities)

def min_intensity(intensities):
    #Get intensities values of vowels in the ROI
    intensitiesvalue = np.delete(np.unique(intensities), np.where(np.unique(intensities) == 0))
    return intensitiesvalue.min()

def intensity_percentile_10(intensities):
    return np.percentile(intensities, 10)

def intensity_percentile_90(intensities):
    return np.percentile(intensities, 90)

def max_intensity(intensities):
    #Get intensities values of vowels in the ROI
    intensitiesvalue = np.delete(np.unique(intensities), np.where(np.unique(intensities) == 0))
    return intensitiesvalue.max()

def intensity_interquartile_range(intensities):
    return (np.percentile(intensities, 75)-np.percentile(intensities, 25))

def intensity_range(intensities):
    return (max_intensity(intensities)-min_intensity(intensities))

def intensitybased_mean_absolute_deviation(intensities,count):
    #Build the centered data matrix of intensities
    intensitiescent = np.where(intensities==0,intensities,intensities-mean_intensity(intensities, count))
    return ((np.sum(abs(intensitiescent)))*(1/count))

def intensitybased_robust_mean_absolute_deviation(intensities):
    #Get the intensities in the interval [10th percentile, 90th percentile]
    intensities10_90 = np.where(((intensities>=intensity_percentile_10(intensities))&(intensities<=intensity_percentile_90(intensities))),intensities,0)
    #Get the number of intensities in the interval [10th percentile, 90th percentile]
    count = (intensities10_90 != 0).sum()
    #Get the mean of these intensities
    mean = mean_intensity(intensities10_90, count)
    #Build the centered data matrix of intensities in the interval [10th percentile, 90th percentile]
    intensities10_90cent = np.where(intensities10_90==0,intensities10_90,intensities10_90-mean)
    return ((np.sum(abs(intensities10_90cent)))*(1/count))

def intensitybased_median_absolute_deviation(intensities,count):
    #Build the matrix that measures dispersion from the median intensity 
    intensitiesmed = np.where(intensities==0,intensities,intensities-median_intensity(intensities))
    return ((np.sum(abs(intensitiesmed)))*(1/count))

def intensitybased_coefficient_variation(intensities,count):
    return(np.sqrt(intesity_variance(intensities,count))/mean_intensity(intensities, count))

def intensitybased_quartile_coefficient_dispersion(intensities):
    return (np.percentile(intensities, 75)-np.percentile(intensities, 25))/(np.percentile(intensities, 75)+np.percentile(intensities, 25))

def intensitybased_energy(intensities):
    return np.sum(intensities**2)

def root_mean_square_intensity(intensities,count):
    return np.sqrt(intensitybased_energy(intensities)/count)

print(intensity_kurtosis(intensities,count))

-0.3546204806878337


Now, I will test the functions implemented above and export the results as a CSV file.

### Note : 
The variables ***intensities*** and ***count*** have been defined in the first section. They represent respectively the set of intensities of voxels included in the ROI and the number of elements of this set.

In [94]:
with open('results.csv','w',newline='') as csvfile:
    writer=csv.writer(csvfile)
    
    writer.writerow(['Mean intensity', 'Intensity variance', 'Intensity skewness', 'Intensity kurtosis', 'Median intensity ','Minimum intensity','10th intensity percentile','90th intensity percentile','Maximum intensity','Intensity interquartile range','Intensity range','Intensity-based mean absolute deviation','Intensity-based robust mean absolute deviation','Intensity-based median absolute deviation','Intensity-based coefficient of variation','Intensity-based quartile coefficient of dispersion','Intensity-based energy','Root mean square intensity'])
    writer.writerow([mean_intensity(intensities, count),intesity_variance(intensities,count), intensity_skewness(intensities,count), intensity_kurtosis(intensities,count) , median_intensity(intensities), min_intensity(intensities), intensity_percentile_10(intensities),intensity_percentile_90(intensities),max_intensity(intensities),intensity_interquartile_range(intensities),intensity_range(intensities),intensitybased_mean_absolute_deviation(intensities,count),intensitybased_robust_mean_absolute_deviation(intensities),intensitybased_median_absolute_deviation(intensities,count),intensitybased_coefficient_variation(intensities,count), intensitybased_quartile_coefficient_dispersion(intensities),intensitybased_energy(intensities),root_mean_square_intensity(intensities,count)])

In [95]:
#Print the CSV file 
r = pd.read_csv("results.csv",encoding = "ISO-8859-1" )
r.head()

,Mean intensity,Intensity variance,Intensity skewness,Intensity kurtosis,Median intensity,Minimum intensity,10th intensity percentile,90th intensity percentile,Maximum intensity,Intensity interquartile range,Intensity range,Intensity-based mean absolute deviation,Intensity-based robust mean absolute deviation,Intensity-based median absolute deviation,Intensity-based coefficient of variation,Intensity-based quartile coefficient of dispersion,Intensity-based energy,Root mean square intensity
0,2.148649,3.045471,1.083821,-0.35462,1.0,1,1.0,4.0,6,3.0,5,1.552228,1.113834,1.148649,0.812198,0.6,567,2.768061
